In [2]:
from pymatgen.core import Structure, Element, Lattice, PeriodicSite
from pprint import pprint
import numpy as np
from pydash import get

In [3]:
from pymatgen.core import Structure
from pymatgen.analysis.diffusion.neb.full_path_mapper import MigrationGraph

/Users/tashalewis/opt/anaconda3/envs/cms/lib/python3.9/site-packages/pymatgen/analysis/path_finder.py:37: FutureWarning: This code has been superseded by pymatgen.analysis.neb in the separate add-on packagepymatgen-diffusion. This module here is retained for backwards compatibility. It will be removed from2022.1.1.
  warnings.warn(


In [4]:
from maggma.stores.advanced_stores import MongograntStore
from maggma.stores.compound_stores import ConcatStore

/Users/tashalewis/opt/anaconda3/envs/cms/lib/python3.9/site-packages/maggma/utils.py:20: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import VoronoiNN, CrystalNN
vnn = VoronoiNN()
cnn = CrystalNN()

In [6]:
from pymatgen.analysis.structure_matcher import StructureMatcher
sm = StructureMatcher()

In [9]:
# material_id and task_ids
materials = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","materials_2022",key="task_id")
materials.connect()

# "material_ids" & "host_material_ids" & "insertion_material_ids" 
sgroup = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","sgroups_2022",key="_id")
sgroup.connect()

# "material_ids" & "host_structure" & "battery_id" & "entries_composition_summary" 
insert_elec = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","insertion_electrodes_2022",key="_id")
insert_elec.connect()

# "battery_id" & "migration_graph" & "matrix_supercell_structure" & "inserted_ion_coords
mig_graph = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","migration_graph_2022",key="task_id")
mig_graph.connect()

aneba_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv", "approx_neb_analysis", key="wf_uuid")
aneba_store.connect()

elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","vw_elec",key="battid")
elec_store.connect()

# "battery_id" & "host_structure" & "material_ids" & "migration_graph" & "inserted_ion_coords"
rank_elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","rank_electrodes_2022")
rank_elec_store.connect()

In [ ]:
rank_elec_list = list(rank_elec_store.query({'battery_id':{'$regex':'Mg'}}))
rank_elec_list

In [14]:
aneba_doc_list = list(aneba_store.query({'battid':{'$regex':'Mg'}}))

In [24]:
for doc in aneba_doc_list:
    battid = doc['battid']  # Old battery id
    wf_uuid = doc['wf_uuid']
    hop_key = list(doc['data'].keys())
    struct = Structure.from_dict(doc['host']['input'])
    elements = list(struct.composition.as_dict().keys())
    #print(wf_uuid)
    #print(hop_key)

In [25]:
def process(wf_uuid, hop_key):
    doc = aneba_store.query_one({"wf_uuid":wf_uuid})
    structs = [Structure.from_dict(s) for s in doc["data"][hop_key]["output_structures"]]
    indexes = doc["data"][hop_key]["relax_site_indexes"]
    vols = [sum([v["volume"] for v in vnn.get_voronoi_polyhedra(struct,i).values()]) for i,struct in zip(indexes,structs)]
    
    cns = []
    for n,struct in zip(indexes,structs):
        # crystal nn
        cn = cnn.get_cn(struct,n)
        cns.append(cn)
    
    return {"wf_uuid":wf_uuid,
            "vv":vols,
            "cn":cns,
            "energies":doc["data"][hop_key]["energies"],
            "wi_sites":doc["data"][hop_key]["wi_sites"],
            "structs":doc["data"][hop_key]["output_structures"],
            "doc":doc
           }

In [26]:
process('040f2cd0-0577-407d-bc39-7bb709146734', '0+1')

TypeError: 'NoneType' object is not subscriptable